<font color="#dd3542">Importing the required Libraries</font>

In [31]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn import svm,metrics
import random
import math

<font color="#dd3542">Reading data from the csv file and some processing</font>

In [32]:
df = pd.read_csv("letter-recognition.csv")
sam = []
msk = np.random.rand(len(df)) < 0.8
train = df[msk]
test = df[~msk]

<font color="#dd3542">Labelling the classes as +1 and -1 to apply the algorithm</font>

In [33]:
m = train;
N1 = m.sample(10000) #for our sample
m1=m.sample(10000)
m1=m1.replace('A','1')
m1=m1.replace(['B','C','D','E','F','G','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z'],'-1')
m1
X = m1.drop(['target'],axis=1).values   # independent features
y = m1['target']
y1 = y

#replacing A with +1 and everything else with -1


<font color="#dd3542">Processing the initial model for SVM</font>

In [34]:
svc = svm.SVC(kernel='linear', C=1).fit(X, y1)

<font color="#dd3542">The loss function</font>

In [35]:
def l(smodel,zgiven):
	ans=1-int(smodel.predict(zgiven.drop(['target','index'],axis=1).values[0].reshape(1,-1))[0])*int(zgiven['target'].values)
	return ans

<font color="#dd3542">Procedure to find the sampled data</font>

In [36]:
f0 = svc
sam = []
mp=0
mm=0
k =0
flag=0
ztplus1 = m1.sample()
while (mp<(len(m1)/2) and mm<(len(m1)/2)):
  
	zt = m1.sample()
	if (int(zt['target'].values)==1):
		mp=mp+1
	else:
		mm = mm+1
	z_star = m1.sample()
	zt.insert(0, 'index', 0)
	z_star.insert(0, 'index', 0)
	if (flag==1):
		zt = ztplus1
	p1 = math.exp(-l(f0,zt))
	p2 = math.exp(-l(f0,z_star))
	P= p1/p2

	yt =int(zt['target'].values)
	y_star =int(z_star['target'].values)
	if (k==5):
		ztplus1 = z_star
		flag =1

		P_double_dash = 1.2*P
		if (P_double_dash>1):
		  sam.append(z_star.index)
		else:
		   ran_p = random.uniform(0, 1)
		   if (P_double_dash>ran_p):
		    sam.append(z_star.index)
		    
		   else:
		    k=0
		continue
	
	if ((P==1 and yt==-1 and y_star == -1) or (P==1 and yt==1 and y_star == 1) ):
	
		ind = z_star.index
		x = z_star.drop(['target','index'],axis=1).values[0].reshape(1,-1)
		x1 = zt.drop(['target','index'],axis=1).values[0].reshape(1,-1)
		P_dash =  math.exp(-y_star)*int(f0.predict(x)[0])/(-yt*int(f0.predict(x1)[0]))
		ran_p = random.uniform(0, 1)
		if (P_dash>ran_p):
		  sam.append(z_star.index)
		  k = k+1
		else:
		    k=0
	if ((P==1 and yt*y_star == -1) or (P<1) ):
		ind = z_star.index
    

		P_dash = P
		ran_p = random.uniform(0, 1)
		if (P_dash>ran_p):
		  sam.append(z_star.index)
    
		  k = k+1
		else:
		    k=0

<font color="#dd3542">Final sampled dataset to be used</font>

In [37]:
tobeused=[]
for i in range(len(sam)):
  tobeused.append(int(sam[i][0]))
findataset=df.iloc[tobeused]

<font color="#dd3542">Training SVM</font>

In [38]:
X_2 = findataset.drop(['target'],axis=1).values   # independent features
y_2 = findataset['target']
sv_neww = svm.SVC(kernel='linear', C=1).fit(X_2, y_2)

<font color="#dd3542">Accuracy calculation</font>

In [39]:
X_orig=df.drop(['target'],axis=1).values
Y_orig=df['target']
y_pred = sv_neww.predict(X_orig)
print("Accuracy:",metrics.accuracy_score(Y_orig, y_pred))

Accuracy: 0.84805
